# RAG 应用评估

RAG 应用评估是一个复杂的问题，完整的评估有很多指标。

## 1. 相关开源项目

### 1.1 [ragas](https://github.com/explodinggradients/ragas)

开源框架 ragas 有如下指标。

| 阶段 | 指标名称 | 指标描述 | 评估方法 | 是否需要参考答案(`ground truth`) |
| --- | --- | --- | --- | --- | 
| 检索 | 上下文召回 Context Recall | 检索到的上下文`context`与参考答案`ground truth`的一致性程度 | LLM | 是 |
| 检索 | 上下文精确度 Context Precision | 评估检索到的上下文`context`中，和参考答案`ground truth`一致的结果是否靠前 | LLM | 是 |
| 检索 | 上下文相关性 Context Relevancy | 评估检索到的上下文`context`中，和问题`question`的相关程度 | LLM | 否 |
| 生成 | 回答相关性 Answer Relevancy | 问题`question` 和答案 `answer`的相关性 | LLM | 否 |
| 生成 | 回答语义相似性 Answer semantic similarity | 答案 `answer`和参考答案`ground truth`的语义相似度 | 交叉编码器 | 是 |
| 生成 | 回答正确性 Answer Correctness | 答案 `answer`和参考答案`ground truth` 在事实方面的一致性 | LLM | 是 |
| 生成 | 回答忠诚度 Answer Faithfulness | 答案 `answer` 和上下文 `context`的事实一致性 | LLM | 否 |
| 生成 | 回答批评 Answer Critique | 对答案 `answer` 在指定的 Prompt 进行批评以识别有害内容 | LLM | 否 |

当数据集中没有人工标注的参考答案时，就需要使用那些不需要参考答案的指标，例如上下文相关性、回答相关性、回答忠诚度、回答批评等。

如果觉得评估指标过多，在有参考答案时，建议选择如下两个指标：

1. 上下文召回：用来评估检索效果。注意这里如果效果下降，可能原因是检索本身的效果不好，也可能是数据源缺少相关的数据。
2. 回答正确性：用来评估生成效果。

注：开源框架 ragas 中的 Prompt 和交叉编码器都是英文，不适合中文环境。

### 1.2 [tvalmetrics](https://github.com/TonicAI/tvalmetrics)

| Metric Name              | Inputs                                                    | Formula | What does it measure? | Which components does it evaluate? |
| ----------------------- | --------------------------------------------------------- | ------------------------------------------------------------ | ------------------------------------ |----|
| **Answer similarity score** | Question + Reference answer + LLM answer | Score between 0 and 5 | How well the reference answer matches the LLM answer. | All components.                     |
| **Retrieval precision** | Question + Retrieved context                         | (Count of relevant retrieved context) / (Count of retrieved context) | Whether the context retrieved is relevant to answer the given question. | Chunker + Embedder + Retriever    |
| **Augmentation precision** | Question + Retrieved context + LLM answer             | (Count of relevant retrieved context in LLM answer) / (Count of relevant retrieved context) | Whether the relevant context is in the LLM answer. | Prompt builder + LLM                |
| **Augmentation accuracy** | Retrieved context + LLM answer                          | (Count of retrieved context in LLM answer) / (Count of retrieved context) | Whether all the context is in the LLM answer. | Prompt builder + LLM                |
| **Answer consistency** or **Answer consistency binary** | Retrieved context + LLM answer                          | (Count of the main points in the answer that can be attributed to context) / (Count of main points in answer) | Whether there is information in the LLM answer that does not come from the context. | Prompt builder + LLM                |
| **Retrieval k-recall** | Question + Retrieved context + Top k context         | (Count of relevant retrieved context) / (Count of relevant context in top k context) | How well the retrieval system retrieves all of the relevant context. | Chunker + Embedder + Retriever    |




## 2. 数据集准备

评测 RAG 应用，数据集必须有：

- 运行输入：
    - question[str]：问题
- 运行输出
    - answer[str]：RAG 应用给出的回答
    - contexts[list[str]]: 检索到的上下文，顺序则代表相似度。
- 评估输入
    - reference_context[str]: 参考上下文，用于评估检索的正确性。
    - reference_answer[str]: 参考答案，用于评估回答的正确性。

数据集有三种方法准备：

1. 使用开源数据集，比如 ragas 引用的 explodinggradients/fiqa 数据集。但是中文的 RAG 数据集较少。而且开源数据集只能代表 RAG 的通用能力，不能代表 RAG 在特定领域的能力。
2. 人工标注，这种方法需要大量的人力成本，但是可以标注特定领域的数据集。
3. 使用 LLM 自动抽取 QA 对，从而形成数据集。这种方法的优点是成本低，缺点是数据集的质量可能不高。自动标注尽量使用能力较强的 LLM，比如 GPT-4 等。

### 2.1 使用开源数据集

从项目 [m3e-base](https://huggingface.co/moka-ai/m3e-base) 介绍中，可以看到作者收集的众多用来训练 Embedding 模型的中文数据集，从其中挑选问答类数据集处理后可用于 RAG 评估。

筛选后，[dureader_robust](https://huggingface.co/datasets/PaddlePaddle/dureader_robust/viewer/plain_text/train?row=96) 和 [cmrc2018](https://huggingface.co/datasets/cmrc2018) 比较适合用于 RAG 评估。前者的回答过于简略，所以我们选择 cmrc2018 数据集进行评估。

注意：cmrc2018 数据集并没有允许商用，请不要在商业项目中使用。

#### 2.1.1 cmrc 数据集下载和转换

In [1]:
%pip install datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/a0/93/da8a22a292e51ab76f969eb87bda8fd70cc3963b4dd71f67bb92a70a7992/datasets-2.16.0-py3-none-any.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/81/54/84d42a0bee35edba99dee7b59a8d4970eccdd44b99fe728ed912106fc781/filelock-3.13.1-py3-none-any.whl.metadata
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Obtaining dependency information for dill<0.3.8,>=0.3.0 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
  Using cached dill-0.3.7-py3-none-any.whl.

In [2]:
from datasets import load_dataset

cmrc = load_dataset("cmrc2018")
print(cmrc["test"][0])
cmrc

/Users/cynic/src/github.com/ninehills/langeval/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/cynic/src/github.com/ninehills/langeval/.venv/lib/python3.11/site-packages/datasets/load.py:1429: FutureWarning: The repository for cmrc2018 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cmrc2018
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'id': 'TRIAL_800_QUERY_0', 'context': '基于《跑跑卡丁车》与《泡泡堂》上所开发的游戏，由韩国Nexon开发与发行。中国大陆由盛大游戏运营，这是Nexon时隔6年再次授予盛大网络其游戏运营权。台湾由游戏橘子运营。玩家以水枪、小枪、锤子或是水炸弹泡封敌人(玩家或NPC)，即为一泡封，将水泡击破为一踢爆。若水泡未在时间内踢爆，则会从水泡中释放或被队友救援(即为一救援)。每次泡封会减少生命数，生命数耗完即算为踢爆。重生者在一定时间内为无敌状态，以踢爆数计分较多者获胜，规则因模式而有差异。以2V2、4V4随机配对的方式，玩家可依胜场数爬牌位(依序为原石、铜牌、银牌、金牌、白金、钻石、大师) ，可选择经典、热血、狙击等模式进行游戏。若游戏中离，则4分钟内不得进行配对(每次中离+4分钟)。开放时间为暑假或寒假期间内不定期开放，8人经典模式随机配对，采计分方式，活动时间内分数越多，终了时可依该名次获得奖励。', 'question': '生命数耗完即算为什么？', 'answers': {'text': ['踢爆'], 'answer_start': [127]}}


DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

正式项目中，应该以全部的数据作为我们的向量索引的基础数据，单条数据格式如上。我们挑选其中的 context 嵌入到向量索引中。

本次我们就只以 test 的 1000 条数据建立索引。同时以 test 的随机 50 条查询作为验证数据集。

此时我们可以准备两种数据集

1. 用于运行的数据集，即只有 question / reference_answer / reference_contexts 的数据集。
2. 用于评估的数据集，即使用 chain 检索后的数据集，包含全部数据。

前者在启动评估任务的时候还要注册 Provider 用于 RAG 生成，为模拟真实的情况，我们选择 1。那么此处只需要生成用于运行的数据集即可。


In [3]:
test_data = []
for i in cmrc["test"].shuffle(seed=42).select(range(100)):
    test_data.append({
        "question": i["question"],
        "reference_context": i["context"],
        "reference_answer": i["answers"]["text"][0],
    })
import pandas as pd
pd.DataFrame(test_data).head()

,question,reference_context,reference_answer
0,与藤原纪香的婚礼耗资多少日圆？,阵内智则（1974年2月22日－），日本喜剧演员及主持。曾经于日本节目《娱乐之神》表演多场短...,5亿日圆
1,此剧的制作人是谁？,《西南忠魂》是台湾台湾电视公司于1984年（民国73年）3月28日至1985年（民国74年）...,制作人伍宗德
2,达曼的卫星城有哪几座？,达曼（）位于沙特阿拉伯的东部省，是沙特石油工业的重要中心。达曼是东部省最大的城市，达曼港也是...,达曼的卫星城有现代经济中心 Khobar、世界最大的沙特Aramco石油公司所在地札哈兰以及...
3,《This Is Where I Came In》是Bee Gees的第几张原创专辑？,《This Is Where I Came In》是Bee Gees的第20张原创专辑，也是...,第20张
4,卡洛身兼几家骑士团的大团长职位？,卡洛·玛利亚·贝尔纳多·真纳罗（Carlo Maria Bernardo Gennaro，）...,圣乔治康斯坦丁骑士团、圣斐迪南骑士团、圣真纳罗骑士团和弗朗切斯科一世王家骑士团


In [4]:
import json
with open("cmrc-eval-zh.jsonl", "w") as f:
    for i in test_data:
        f.write(json.dumps(i, ensure_ascii=False) + "\n")

In [10]:
!pip install langchain openai tiktoken faiss-cpu

  Using cached faiss_cpu-1.7.4-cp311-cp311-macosx_10_9_x86_64.whl (6.5 MB)

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import getpass
openai_api_key = getpass.getpass("Please input your openai api key: ")

In [7]:
# 正式项目中，应该以全部的数据作为我们的向量索引的基础数据，单条数据格式如上。我们挑选其中的 context 嵌入到向量索引中。
# 本次我们就只以 test 的 1000 条数据建立索引。

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

# 从 cmrc 中提取 context
texts_set = set()
texts = []
metadatas = []
for i in cmrc["test"]:
    if i["context"] not in texts_set:
        # 去重
        texts.append(i["context"])
        metadatas.append({"id": i["id"]})
        texts_set.add(i["context"])
print(texts[0])
print(metadatas[0])
print(len(texts))

基于《跑跑卡丁车》与《泡泡堂》上所开发的游戏，由韩国Nexon开发与发行。中国大陆由盛大游戏运营，这是Nexon时隔6年再次授予盛大网络其游戏运营权。台湾由游戏橘子运营。玩家以水枪、小枪、锤子或是水炸弹泡封敌人(玩家或NPC)，即为一泡封，将水泡击破为一踢爆。若水泡未在时间内踢爆，则会从水泡中释放或被队友救援(即为一救援)。每次泡封会减少生命数，生命数耗完即算为踢爆。重生者在一定时间内为无敌状态，以踢爆数计分较多者获胜，规则因模式而有差异。以2V2、4V4随机配对的方式，玩家可依胜场数爬牌位(依序为原石、铜牌、银牌、金牌、白金、钻石、大师) ，可选择经典、热血、狙击等模式进行游戏。若游戏中离，则4分钟内不得进行配对(每次中离+4分钟)。开放时间为暑假或寒假期间内不定期开放，8人经典模式随机配对，采计分方式，活动时间内分数越多，终了时可依该名次获得奖励。
{'id': 'TRIAL_800_QUERY_0'}
256


In [8]:
# init embedding
embed = OpenAIEmbeddings(openai_api_key=openai_api_key)
embed.embed_query("你好")

[0.0002545917378902969,
 -0.006038765305652746,
 -0.002320892544221213,
 -0.02918419037709719,
 -0.04373818515692064,
 0.013779306647735901,
 -0.0224786842169005,
 -0.00846443094706955,
 -0.01518898731162881,
 -0.01979902314601002,
 0.035534100793988686,
 0.001334275079551899,
 0.004746558418468674,
 -0.002333592292411831,
 -0.008337432999502056,
 -0.016712966407729252,
 0.03423872042494857,
 -0.014884191492408721,
 0.01803374767013455,
 -0.02039591340644498,
 -3.0360511960178458e-05,
 0.001309669186465332,
 0.009372468647049836,
 0.0008993061428860049,
 -0.007721491603381899,
 -0.007937388952437004,
 0.009994759800849974,
 -0.018427441959519623,
 0.00723889837817053,
 -0.00849618066679208,
 0.013245913964100745,
 0.010705950356137722,
 -0.02971758306073234,
 -0.002682837346714411,
 0.011950532663737998,
 -0.01336021239630828,
 -0.01231882699108052,
 -0.001854174062455781,
 0.02076420866511013,
 0.0048926065703987385,
 0.01701776222694934,
 0.010401154536917635,
 0.00952486655665988,
 

In [11]:
# load to vectorstore
vectorstore = FAISS.from_texts(
    texts=texts,
    embedding=embed,
    metadatas=metadatas,
)
vectorstore.similarity_search("你好")

[Document(page_content='张天霖（）台湾男演员，毕业于漳和国中、台北市立建国中学补校、真理大学运动管理学系。主要演艺工作以电视剧为主，另有广告代言与 MV 演出，也有跨行书籍著作及电影导演等。原本是模特儿的张天霖MARCUS，高二的时候拍了周华健的MV「朋友」。当时在倪雅伦的介绍下签给经纪人方登恺Kris Fang uniqe premier model management；于是便与名模孟广美、倪雅伦、赖雅妍、杨祐宁等人开始了模特儿的工作。在结束近8年的模特儿后开始参与电视剧演出，主要以偶像剧为主，著名作品包括《吐司男之吻》、《我的秘密花园》、《男丁格尔》等，自2007年首次参与中视八点档连续剧《豪门本色》的演出后,在台湾的演出便一直以八点档为主。身高182CM', metadata={'id': 'TRIAL_311_QUERY_0'}),
 Document(page_content='林郁智（），艺名纳豆，台湾男演员、主持人。金星娱乐旗下签约艺人（俗称伟忠帮），现是台视 《综艺3国智》、《完全娱乐》的主持人，八大电视台「同学！搞什么鬼！2013年9月26日和主持搭档任家萱（Selina）以《小宇宙33号》首次入围第48届金钟奖综艺节目主持人奖。2016年以公路电影《一路顺风》入围第53届金马奖最佳男配角奖。纳豆说会跟林千又交往，就是之前在一家餐厅用餐，发现隔壁桌的女孩一直盯著他看，餐后他去结帐并过去招呼，「原来她会看我，是因为我们3年前就同上过《国光帮帮忙》过年特别节目，只是当时没讲到话，用餐那天我是去跟朋友谈民宿投资，于是我跟她互留Line，邀她有空来民宿玩」。没想到3个月后，两人真的以男女朋友的身分，去到他澎湖新开幕的民宿度假，一切都是因缘巧合。2016年以「生活习惯不一样」为由宣布分手。广播', metadata={'id': 'TRIAL_924_QUERY_0'}),
 Document(page_content='蔡敏（），籍贯湖北云梦大山乡人，出生地于台湾台北市，台湾三大女高音之一。其父为中华民国空军中将蔡名永。蔡敏曾就学于台北市空军子弟学校、台北市古亭国小、台北市万华女中、台北县新店崇光女中、台北市实践大学音乐系、义大利米兰音乐院、美国华盛顿天主教大学声乐硕士。1977年香港亚洲歌唱比赛季军。1977年与陈威陵、吴函、李

In [12]:
vectorstore.save_local("cmrc-eval-zh.faiss")

### 2.2 使用 LLM 标注数据集

TODO，参考 https://docs.llamaindex.ai/en/stable/examples/evaluation/QuestionGeneration.html

主要方法是先拆分 context，然后对每段 context 使用 LLM 抽取 QA 对，从而形成数据集。

## 3. Embedding 模型评估和微调

可以使用 c-mteb 中 Reranking 方法评估检索模型的效果。仅需要 question 和 context ，负面样本可以自动生成。评估如果性能不佳，可以引入微调方案。

具体参见 [c-mteb](c-mteb/c-mteb.md)，但是这种方法只能评估 Embedding 模型，如果你的检索是混合检索（比如混合了 BM25 + Embedding + Rerank），那么就无法通过这种方法评估检索效果。

## 4. 运行 + 评估

In [ ]:
# 只对检索进行评估，先验证脚本的正确性
!OPENAI_API_KEY="xxxxx" langeval -v run cmrc-eval-zh.yaml --sample 1

# 批量验证
!OPENAI_API_KEY="xxxxx" langeval run cmrc-eval-zh.yaml

# 注意依赖数据集中的 reference_context 字段

```
+---------------------------------------+-------+--------------------+---------------------+-----+--------------------+-----+-----+-----+
| eval                                  | count | mean               | std                 | min | 25%                | 50% | 75% | max |
+=======================================+=======+====================+=====================+=====+====================+=====+=====+=====+
| rag.outputs.retrieval_recall_hit_rate | 100.0 | 0.91               | 0.2876234912646614  | 0.0 | 1.0                | 1.0 | 1.0 | 1.0 |
+---------------------------------------+-------+--------------------+---------------------+-----+--------------------+-----+-----+-----+
| rag.outputs.retrieval_recall_mrr      | 100.0 | 0.8853333333333333 | 0.30599399215070094 | 0.0 | 1.0                | 1.0 | 1.0 | 1.0 |
+---------------------------------------+-------+--------------------+---------------------+-----+--------------------+-----+-----+-----+
| rag.outputs.retrieval_recall_ndgc_10  | 100.0 | 0.9208817691055944 | 0.23084627440062405 | 0.0 | 0.9999159195080202 | 1.0 | 1.0 | 1.0 |
+---------------------------------------+-------+--------------------+---------------------+-----+--------------------+-----+-----+-----+
```

In [ ]:
# 对生成+检索进行评估，测试
!OPENAI_API_KEY="xxxxx" langeval -v run rag-eval.yaml --sample 1

# 注意依赖数据集中的 reference_context、 reference_answer 字段